In [25]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pytz
from datetime import datetime
import yfinance as yf

In [26]:
def get_sp500_tickers():
    res = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    soup = BeautifulSoup(res.content, "html")
    table = soup.find_all("table")[0]
    df = pd.read_html(str(table))
    tickers = list(df[0].Symbol)
    return tickers

tickers = get_sp500_tickers()

In [27]:
period_start = datetime(2010, 1, 1, tzinfo=pytz.utc)
period_end = datetime(2020, 1, 1, tzinfo=pytz.utc)
print(period_start, period_end)

2010-01-01 00:00:00+00:00 2020-01-01 00:00:00+00:00


In [28]:
def get_history(ticker, start_date, end_date, granularity = "1d"):
    df = yf.Ticker(ticker).history(
        start=start_date, 
        end=end_date, 
        interval=granularity, 
        auto_adjust=True
    ).reset_index()
    
    df = df.rename(columns={
        "Date": "datetime",
        "Open": "open",
        "High": "high",
        "Low": "low",
        "Close": "close",
        "Volume": "volume"
    })
    
    if df.empty:
        return pd.DataFrame()
    
    df["datetime"] = df["datetime"].dt.tz_convert(pytz.utc)
    
    df = df.drop(columns = ["Dividends", "Stock Splits"])
    
    df = df.set_index("datetime", drop = True)
    
    return df

In [29]:
for ticker in tickers:
    df = get_history(ticker, start_date=period_start, end_date=period_end)
    print(ticker, df)

MMM                                  open        high         low       close  \
datetime                                                                    
2010-01-04 05:00:00+00:00   53.340328   53.571433   53.070707   53.295391   
2010-01-05 05:00:00+00:00   53.154168   53.430210   52.448010   52.961578   
2010-01-06 05:00:00+00:00   53.847469   54.309679   53.609947   53.712658   
2010-01-07 05:00:00+00:00   53.487977   53.770441   52.717629   53.751183   
2010-01-08 05:00:00+00:00   53.725500   54.129932   53.475137   54.129932   
...                               ...         ...         ...         ...   
2019-12-24 05:00:00+00:00  148.234274  148.566876  146.379986  146.912155   
2019-12-26 05:00:00+00:00  146.720899  146.895523  145.930962  146.837311   
2019-12-27 05:00:00+00:00  146.887184  147.627232  146.030724  147.394409   
2019-12-30 05:00:00+00:00  147.394444  148.808031  146.005815  146.205383   
2019-12-31 05:00:00+00:00  145.706463  146.837325  145.573430  146.69596

KeyboardInterrupt: 